In [ ]:
%matplotlib inline

In [163]:
import pandas as pd
import cPickle as pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot
import sklearn as sk
import os
from osgeo import ogr
import geopandas as gpd
import operator
import json
from shapely.geometry import Point, shape

In [ ]:
df = pd.read_excel('../NIJ2013_JAN01_DEC31.xlsx')

In [ ]:
df.head()

In [ ]:
col_uniques = []
for i in df.columns:
    col_uniques.append([df[i].unique(), i])

In [ ]:
filter(lambda x: x[1] == 'CASE DESC', col_uniques)

In [ ]:
df[df['final_case_type'] == 'DISTP ']['CASE DESC'].unique()

In [ ]:
for i in df.columns[:-4]:
    df[i] = df[i].str.rstrip()

In [21]:
excels = []
e = os.listdir('../data/')
for i in e:
    if i.endswith('_Data'):
        files = list(os.walk('../data/' + i))[0]
        excel = filter(lambda x: x.endswith('.xlsx'), files[2])
        excels.append(files[0] + '/' + excel[0])
    else:
        pass

In [27]:
dataframes = []
for i in excels:
    print 'reading %s' % i
    dataframes.append(pd.read_excel(i))
    print 'read %s' % i
bigDF = pd.concat(dataframes)

reading ../data/010114_123114_Data/NIJ2014_JAN01_DEC31.xlsx
read ../data/010114_123114_Data/NIJ2014_JAN01_DEC31.xlsx
reading ../data/120116_123116_Data/NIJ2016_DEC01_DEC31.xlsx
read ../data/120116_123116_Data/NIJ2016_DEC01_DEC31.xlsx
reading ../data/110116_113016_Data/NIJ2016_NOV01_NOV30.xlsx
read ../data/110116_113016_Data/NIJ2016_NOV01_NOV30.xlsx
reading ../data/010115_123115_Data/NIJ2015_JAN01_DEC31.xlsx
read ../data/010115_123115_Data/NIJ2015_JAN01_DEC31.xlsx
reading ../data/080116_083116_Data/NIJ2016_AUG01_AUG31_USE.xlsx
read ../data/080116_083116_Data/NIJ2016_AUG01_AUG31_USE.xlsx
reading ../data/100116_103116_Data/NIJ2016_OCT01_OCT31.xlsx
read ../data/100116_103116_Data/NIJ2016_OCT01_OCT31.xlsx
reading ../data/030112_123112_Data/NIJ2012_MAR01_DEC31.xlsx
read ../data/030112_123112_Data/NIJ2012_MAR01_DEC31.xlsx
reading ../data/090116_093016_Data/NIJ2016_SEP01_SEP30.xlsx
read ../data/090116_093016_Data/NIJ2016_SEP01_SEP30.xlsx
reading ../data/010113_123113_Data/NIJ2013_JAN01_DEC31.x

In [29]:
bigDF.to_csv('../data/cleanData/all_crim_data.csv')

In [30]:
burg = bigDF[bigDF['CATEGORY'] == 'BURGLARY']
burg_counts = burg[['CATEGORY', 'occ_date']].groupby('occ_date').count()

In [ ]:
burg_counts.resample('2Q', 'sum').plot()

In [ ]:
#fig, ax = pyplot.subplots()
#weekly_sample.plot()

In [ ]:
#driver = ogr.GetDriverByName('ESRI Shapefile')
#gridfile = "../data/shapefiles/testfinalclip.shp"
#gridSource = driver.Open(gridfile, 0) # 0 means read-only. 1 means writeable.
#gridLayer = gridSource.GetLayer()
#gridPoints = []
#geoms = []
#for feature in gridLayer:
#    geom = feature.GetGeometryRef()
#    geoms.append(geom)
#    p = geom.GetEnvelope()
#    print p
#    gridPoints.append(p)

In [2]:
g1 = gpd.GeoDataFrame.from_file("../data/shapefiles/Portland_Police_Districts.shp")
g2 = gpd.GeoDataFrame.from_file("../data/shapefiles/polyGrid.shp")

In [12]:
data = []
for index, dist in g1.iterrows():
    for index2, grid in g2.iterrows():
        if dist['geometry'].intersects(grid['geometry']):
            data.append({'geometry': dist['geometry'].intersection(grid['geometry']), 'district':dist['DISTRICT'], 
                         'precinct': dist['PRECINCT']})


In [20]:
#geoDF = gpd.GeoDataFrame(data, columns=['district', 'geometry', 'precinct'])
geoDF.to_file('../data/shapefiles/intersections.shp')

In [119]:
squares = []
for i in range(len(bigDF)):
    if i % 10000 == 0:
        print 'on row %d, %d percent complete' % (i, float(i)/len(bigDF))
    coord = bigDF.iloc[i][['x_coordinate', 'y_coordinate']]
    pt = Point(coord['x_coordinate'], coord['y_coordinate'])
    g = geoDF[geoDF.geometry.contains(pt) == True]['geometry']
    squares.append({'df_index':i, 'square_index': g.index.values[0]})

on row 0, 0 percent complete
39
101
232
262
327
488
704


KeyboardInterrupt: 

In [ ]:
#data = []
#for i in range(len(g1)):
#    print i
#    for y in range(len(g3)):
#        if y % 1000 == 0:
#            print y
#        if g1.iloc[i]['geometry'].intersects(g2.iloc[y]['geometry']):
#            data.append({'geometry': g1.iloc[i]['geometry'].intersection(g2.iloc[y]['geometry']), 'district':g1.iloc[i]['DISTRICT'], 
#                       'precinct': g1.iloc[i]['PRECINCT']})

In [117]:
coord = bigDF.iloc[-1][['x_coordinate', 'y_coordinate']]
pt = Point(coord['x_coordinate'], coord['y_coordinate'])
g = geoDF[geoDF.geometry.contains(pt) == True]['geometry']

In [122]:
bigDF[926740:926744]

,CATEGORY,CALL GROUPS,final_case_type,CASE DESC,occ_date,x_coordinate,y_coordinate,census_tract
120462,BURGLARY,PROPERTY CRIME,BURGP,BURGLARY - PRIORITY *H ...,2016-07-29,7644019,686939,5100
120463,BURGLARY,PROPERTY CRIME,BURGP,BURGLARY - PRIORITY *H ...,2016-07-30,7669140,678601,1601
120464,BURGLARY,PROPERTY CRIME,BURGP,BURGLARY - PRIORITY *H ...,2016-07-30,7671422,695799,7700
120465,BURGLARY,PROPERTY CRIME,BURGP,BURGLARY - PRIORITY *H ...,2016-07-31,7685840,684298,9302


In [128]:
sq = filter(lambda x: len(x) != 0, squares)

In [134]:
sq[0].index.values[0]

24479

In [141]:
ldf = bigDF[10:20]
ldf.iloc[9][['x_coordinate', 'y_coordinate']]

x_coordinate    7646563
y_coordinate     684232
Name: 19, dtype: object

In [150]:
sq = pickle.load(open('../data/cleanData/testmatched.pkl', 'rb'))

In [155]:
GEO_DF_PATH = "../data/shapefiles/intersections.shp"

def matchPoints(bigDF):
    geoDF =  gpd.GeoDataFrame.from_file(GEO_DF_PATH)
    squares = []
    for i in range(len(bigDF)):
        if i % 10000 == 0:
            print 'on row %d, %f percent complete' % (i, float(i)/float(len(bigDF)))
        coord = bigDF.iloc[i][['x_coordinate', 'y_coordinate']]
        pt = Point(coord['x_coordinate'], coord['y_coordinate'])
        g = geoDF[geoDF.geometry.contains(pt) == True]['geometry']
        if len(g) > 0:
            squares.append({'df_index':i, 'square_index': g.index.values[0]})
        else:
            squares.append({'df_index':i, 'square_index': None})
        print 'COMPLETED %d rows, %f percent complete' % (i+1, float(i+1)/float(len(bigDF)))
    return squares

In [153]:
print 'reading pdf'
#pdf = pd.read_csv('../data/cleanData/all_crim_data.csv')
chunk_size = 2
splits = range(0, len(bigDF), chunk_size)
#chunks = map(lambda x: x + chunk_size, splits)
print 'splitting df'
df_list = map(lambda x: bigDF[x:x+chunk_size], splits)
print 'starting matching process'
res = map(lambda x: matchPoints(x), df_list[:2])
print 'completed match'
matched = reduce(operator.add, res)
pklPath = '../data/cleanData/testmatched.pkl'
print 'saving %d matches in %s' % (len(matched), pklPath)
#pickle.dump(matched, open(pklPath, 'wb'))

reading pdf
splitting df
starting matching process


NameError: global name 'matchPoints' is not defined

In [156]:
res = map(lambda x: matchPoints(x), df_list[:2])
print 'completed match'
matched = reduce(operator.add, res)

on row 0, 0.000000 percent complete
COMPLETED 1 rows, 0.500000 percent complete
COMPLETED 2 rows, 1.000000 percent complete
on row 0, 0.000000 percent complete
COMPLETED 1 rows, 0.500000 percent complete
COMPLETED 2 rows, 1.000000 percent complete
completed match


NameError: name 'operator' is not defined

In [159]:
matched = reduce(operator.add, res)

In [164]:
json.dump(matched, open('testmatched.json', 'wb'))

In [166]:
json.load(open('testmatched.json'))

[{u'df_index': 0, u'square_index': 24479},
 {u'df_index': 1, u'square_index': 5453},
 {u'df_index': 0, u'square_index': 137846},
 {u'df_index': 1, u'square_index': 115292}]

In [167]:
len(bigDF)/13

71288